In [4]:
BASE_DIR = '../'*3
RUN_DIR = BASE_DIR + 'code/CNN/cnn_regmodel/cnn_reg_model.ipynb'

In [5]:
%run {RUN_DIR}

In [ ]:
class CnnExtModel(CnnRegModel):
    macros ={}
    
    def __init__(self, name, dataset, hconfigs, show_maps=False,\
                l2_decay = 0, l1_decay = 0, dump_structure = False):
        self.dump_structure = dump_structure
        self.layer_index = 0
        self.layer_depth=0
        self.param_count = 0
        #
#         self.modules = nn.Sequential()
#         self.modules = nn.Sequential()
        self.modules = {}
        self.net = []
        self.name = name
        self.macro =[]
        self.idx = 1
        #
        super(CnnExtModel, self).__init__(name, dataset, hconfigs, show_maps,l2_decay, l1_decay)
        if self.dump_structure :
            print('Total parameter count : {}'.format(self.param_count))

In [ ]:
def cnn_ext_alloc_layer_param(self, input_shape, hconfig):
    layer_type = get_layer_type(hconfig)
    if layer_type in ['serial', 'parallel', 'loop','add','custom']:
        if self.dump_structure:
            dump_str = layer_type
            
            if layer_type =='custom':
                name = get_conf_param(hconfig, 'name')
                dump_str += ' '+name
            print('{:>{width}}{}'.format('', dump_str, width = self.layer_depth*2))
        self.layer_depth += 1;
    pm, output_shape = super(CnnExtModel, self).alloc_layer_param(input_shape, hconfig)
    
    if layer_type in ['serial', 'parallel', 'loop', 'add', 'custom']:
        self.layer_depth -= 1;
        
    elif self.dump_structure:
        self.layer_index +=1
        pm_str ='';
        if layer_type == 'full':
#             print(x)
            ph, pw = pm[0].weight.shape
            pm_count = torch.prod(torch.tensor(pm[0].weight.shape)) + torch.tensor(pm[0].bias.shape[0])
            self.param_count += pm_count
            pm_str = 'pm :{}x{}+{}={}'.format(ph,pw,torch.tensor(pm[0].bias.shape[0]),pm_count)

        elif layer_type == 'conv':
                    
            ychn,xchn,kh,kw = pm[0].weight.shape
            pm_count= torch.prod(torch.tensor(pm[0].weight.shape))+torch.tensor(pm[0].bias.shape[0])
            self.param_count += pm_count
            
            
            pm_str = 'pm : {}x{}x{}x{}+{} = {}'.format(ychn, xchn, kh, kw, torch.tensor(pm[0].bias.shape[0]), pm_count)
        print('{:>{width}}{}: {}, {}=>{}{}'.format('', self.layer_index, layer_type, input_shape, output_shape, pm_str,width=self.layer_depth*2))
    
    return pm, output_shape

CnnExtModel.alloc_layer_param = cnn_ext_alloc_layer_param

In [ ]:
def cnn_ext_alloc_parallel_layer(self, input_shape, hconfig):
#     pm_hiddens_parallel = nn.Sequential()
    pm_hiddens_parallel = []

    num_parallel=0
    output_shape =None
    
    if not isinstance(hconfig[1], dict): hconfig.insert(1,{})
            
        
    for bconfig in hconfig[2:]:

        bpm, bshape = self.alloc_layer_param(input_shape, bconfig)
        if bpm!=None:
            pm_hiddens_parallel.append(bpm[0])

#             pm_hiddens_parallel.add_module('parallel_{0}'.format(self.layer_index), bpm[0])
            num_parallel += 1
        if output_shape:
            assert output_shape[1:] == bshape[1:]
            output_shape[0] += bshape[0]
            
        else:
            output_shape = bshape
            
        
    return [pm_hiddens_parallel], output_shape


CnnExtModel.alloc_parallel_layer = cnn_ext_alloc_parallel_layer


In [ ]:
def cnn_ext_alloc_serial_layer(self, input_shape, hconfig):
#     pm_hiddens_serial =nn.Sequential()
    pm_hiddens_serial =[]
    prev_shape = input_shape
    if not isinstance(hconfig[1], dict) : hconfig.insert(1,{})
    num_serial=0
    for sconfig in hconfig[2:]:
        
        pm_hidden, prev_shape = self.alloc_layer_param(prev_shape,sconfig)
#         print(pm_hidden)
#         exit()
        if pm_hidden != None:
            pm_hiddens_serial.append(pm_hidden[0])
#             pm_hiddens_serial.add_module('serial_{0}'.format(self.layer_index), pm_hidden[0])
#             self.serial.add_module('{0}'.format(self.num), pm_hidden[0])
            num_serial += 1
    return [pm_hiddens_serial], prev_shape





CnnExtModel.alloc_serial_layer = cnn_ext_alloc_serial_layer


In [ ]:
def cnn_ext_alloc_add_layer(self, input_shape, hconfig):
    if not isinstance(hconfig[1], dict): hconfig.insert(1, {})
    
    bpm, output_shape = self.alloc_layer_param(input_shape, hconfig[2])
    pm_hiddens = [bpm] 
    
    for bconfig in hconfig[3:]:
        bpm, bshape =self.alloc_layer_param(input_shape, bconfig)
        pm_hiddens.append(bpm)
        check_add_shapes(output_shape, bshape)
        
    if get_conf_param(hconfig, 'x', True):
        check_add_shape(output_shape, input_shape)
        
    pm = {'pms':pm_hiddens}
    
    for act in get_conf_param(hconfig, 'actions', ''):
        if act == 'B':
            bn_config = ['batch_normal', {'rescale':True}]
            pm['bn'], _ =self.alloc_batch_normal_param(output_shape, bn_config)
            
    return pm, output_shape

CnnExtModel.alloc_add_layer = cnn_ext_alloc_add_layer

In [ ]:
# def cnn_ext_forward_add_layer(self, x, hconfig, pm):
#     y, baux = self.forward_layer(x, hconfig[2], pm['pms'][0])
#     bauxes, bchns, aux_bn = [baux], [y.shape[-1]], [] 
    
#     for n, bconfig in enumerate(hconfig[3:]):
#         by, baux = self.forward_layer(x, bconfig, pm['pms'][n+1])
#         y += tile_add_result(by, y.shape[-1], by.shape[-1])
        
#         bauxes.append(baux)
#         bchns.append(by.shape[-1])
        
        
#     if get_conf_param(hconfig, 'x', True):
#         y += tile_add_result(x, y.shape[-1], x.shape[-1])
        
#     for act in get_conf_param(hconfig, 'actions',''):
#         if act == 'A' : y = self.activate(y, hconfig)
#         if act == 'B':
#             y, aux_bn = self.forward_batch_normal_layer(y, None, pm['bn'])
            
            
#     return y, [y, bauxes, bchns, aux_bn, x.shape]


# CnnExtModel.forward_add_layer = cnn_ext_forward_add_layer

In [ ]:
# def cnn_ext_backprop_add_layer(self, G_y, hconfig, pm, aux):
#     y, bauxes, bchns, aux_bn, x_shape = aux
    
#     for act in reversed(get_conf_param(hconfig, 'actions','')):
#         if act == 'A' : G_y = self.activate_derv(G_y, y, hconfig)
#         if act == 'B':
#             G_y = self.backprop_batch_normal_layer(G_y, None, pm['bn'], aux_bn)
            
#     G_x = np.zeros(x_shape)
    
#     for n, bconfig in enumerate(hconfig[2:]):
#         G_by = merge_add_grad(G_y, G_y.shape[-1], bchns[n])
#         G_x += self.backprop_layer(G_by, bconfig, pm['pms'][n], bauxes[n])
        
#     if get_conf_param(hconfig, 'x', True):
#         G_x += merge_add_grad(G_y, G_y.shape[-1], x_shape[-1])
        
#     return G_x


# CnnExtModel.backprop_add_layer = cnn_ext_backprop_add_layer

In [ ]:
def check_add_shapes(yshape, bashpe):
    assert yshape[:-1] == bshape[:-1]
    assert yshape[-1] % bshape[-1] == 0
    
def tile_add_result(by, ychn, bchn):
    if ychn == bchn : return by
    times = ychn // bchn
    return np.tile(by, times)

def merge_add_grad(G_y, ychn, bchn):
    if ychn == bchn: return G_y
    
    times = ychn // bchn
    split_shape = G_y.shape[:-1] + tuple([times, bchn])
    return np.sum(G_y.reshape(split_shape), axis = -2)

In [ ]:
def cnn_ext_alloc_loop_layer(sefl, input_shape, hconfig):
    pm_hiddens = []
    prev_shape = input_shape
    
    if not isinstance(hconfig[1], dict): hconfig.insert(1,{})
        
    for n in range(get_conf_param(hconfig, 'repeat',1)):
        pm_hidden, prev_shape = self.alloc_layer_param(prev_shape, hconfig[2])
        pm_hiddens.append(pm_hidden)
        
    return {'pms':pm_hiddens},prev_shape


def cnn_ext_forward_loop_layer(self, x, hconfig, pm):
    hidden = x
    aux_layers = []
    
    for n in range(get_conf_param(hconfig, 'repeat', 1)):
        hidden, aux = self.forward_layer(hidden, hconfig[2], pm['pms'][n])
        aux_layers.append(aux)
        
    return hidden, aux_layers


CnnExtModel.alloc_loop_layer = cnn_ext_alloc_loop_layer
CnnExtModel.alloc_forward_layer = cnn_ext_forward_loop_layer


In [1]:
import torch.nn as nn

In [ ]:
def cnn_ext_alloc_custom_layer(self, input_shape, hconfig):
    
    name = get_conf_param(hconfig, 'name')
    args = get_conf_param(hconfig, 'args',{})
    macro = CnnExtModel.get_macro(name, args)
    print(len(macro))
    pm_hidden, output_shape = self.alloc_layer_param(input_shape, macro)
    assert isinstance(macro[1], dict)
    
#     module=self.get_module(name, pm_hidden, macro):
#     print()    
#     self.modules.add_module(name+'_{0}'.format(self.idx), pm_hidden[0])
    col = name+'_{0}'.format(self.idx)
    self.modules[col]= pm_hidden[0]
    
    if len(macro)==self.idx+1:
        pass
    elif name.split('_')[-1] == 'preproc' and 'postproc':
#         print(type(name),name,'namenamenamenamenamenamenamenamenamenamenamename')
#         print(x)

        self.net.append(nn.Sequential(*pm_hidden[0]))
    
    
    else:
        mo_dict=self.get_module_dict(pm_hidden[0], col)
        B=Block(pm_hidden[0], col, mo_dict)
        self.net.append(B)
    self.macro.append(macro)
    
    self.idx += 1
    
    return self.net, output_shape

def cnn_ext_forward_custom_layer(self, x, hconfig, pm):
    return self.forward_layer(x, pm['macro'], pm['pm'])



CnnExtModel.alloc_custom_layer = cnn_ext_alloc_custom_layer
CnnExtModel.forward_custom_layer = cnn_ext_forward_custom_layer


In [ ]:
def cnn_ext_get_module_dict(self,pm, col):
    dic={}
    idx = 1
    for i in pm:
        if not isinstance(i,list):
            print(type(col),col)
            print(idx)
            print(i)
            dic[col+'_'+str(idx)]=i
            idx += 1
        else:
            for j in i:
                if not isinstance(j, list):
                    dic[col+'_'+str(idx)]=j
                    idx +=1
                else:
                    k,m = j
                    dic[col+'_'+str(idx)]=k
                    idx +=1
                    dic[col+'_'+str(idx)]=m
                    idx +=1
    return dic
    
CnnExtModel.get_module_dict = cnn_ext_get_module_dict


In [ ]:
class Block(nn.Module):
    def __init__(self, modules, col, module_dict):
        
        super(Block, self).__init__()
        
        self.col = col
        self.modules = modules
        
        setattr(self, self.col, nn.Sequential())
        method = getattr(self, self.col)
        method.add_module('dict', nn.ModuleDict(module_dict))
        
    def forward(self,x):
        
        cat_modules=[]
        idx = 1
        print(self.modules)
        for i in self.modules:
            if not isinstance(i,list):
                print(i)
                print(idx)
                print(self.col)
                out,idx = self.get_output(idx,x)
                cat_modules.append(out)
            else:
                out_val = None
                for j in i:
                    if not isinstance(j, list):
                        out,idx = self.get_output(idx, x)
                        out_val=out
                    else:
                        out,idx = self.get_output(idx,out_val)
                        cat_modules.append(out)
        print(torch.cat(cat_modules,dim=1).shape,'222222222222222222')
        return torch.cat(cat_modules,dim=1)                
            
    def get_output(self,idx,x):
        out_method = getattr(self, self.col)
        if isinstance(x, list):
            val1, val2=x
            val1 = out_method.dict[self.col+'_'+ str(idx)](x)
            idx += 1
            val2 = out_method.dict[self.col+'_' + str(idx)](x)
            idx += 1
            print(val1.shape,val2.shape,'1111111111111111111111111111111')
            return torch.cat([val1,val2],dim=1),idx
        
        else:
            out = out_method.dict[self.col+'_' + str(idx)](x)
            print(out.shape)
            idx += 1

            return out,idx


In [ ]:
def cnn_ext_set_macro(name, config):
    CnnExtModel.macros[name] = config
    
def cnn_ext_get_macro(name, args):
    
    restored = copy.deepcopy(CnnExtModel.macros[name])
    replace_arg(restored, args)
    
    return restored

def replace_arg(exp, args):
    if isinstance(exp, (list, tuple)):
        for n, term in enumerate(exp):
            if isinstance(term, str) and term[0] == '#':
                if term[1] == '#' :exp[n]= term[1:]
                elif term in args: exp[n]= args[term]
                    
            else:
                replace_arg(term, args)
                
    elif isinstance(exp, dict):
        for key in exp:
            if isinstance(exp[key], str) and exp[key][0] == '#':
                if exp[key][1]=='#' :exp[key] = exp[key][1:]
                elif exp[key] in args : exp[key] = args[exp[key]]
                    
            else:
                replace_arg(exp[key], args)
                
CnnExtModel.set_macro = cnn_ext_set_macro
CnnExtModel.get_macro = cnn_ext_get_macro

In [ ]:
def cnn_ext_alloc_conv_layer(self, input_shape, hconfig):
    pm, output_shape = super(CnnExtModel, self).alloc_conv_layer(input_shape, hconfig)
    
    # 상속을 통하여 기존의 것들을 사용하고 추가되는 부분을 추가
    act_func = get_conf_param(hconfig, 'actions','LA')
    
    for act in act_func:
        
        if act == 'L':
            input_shape = output_shape
        elif act == 'B':
            bn_config=['batch_noram', {'rescale':False}]
            pm['bn'], _ = self.alloc_batch_normal_layer(input_shape, bn_config)
            
            
    xchn, xh, xw = input_shape
    
    ychn= get_conf_param(hconfig, 'chn')
    
    output_shape = eval_stride_shape(hconfig, True, ychn, xh, xw)
#     self.serial.add_module('{0}'.format(self.num),pm[0])
    return pm, output_shape


CnnExtModel.alloc_conv_layer = cnn_ext_alloc_conv_layer

In [ ]:
def cnn_ext_forward_conv_layer(self, x, hconfig, pm):
    y = x
    x_flat, k_flat, relu_y, aux_bn = None, None, None, None
    
    for act in pm['actions']:
        if act == 'L':
            mb_size, xh, xw, xchn = y.shape
            kh, kw, _, ychn = pm['k'].shape
            x_flat = get_ext_regions_for_conv(y, kh, kw)
            k_flat = pm['k'].reshape([kh*kw*xchn, ychn])
            conv_flat = np.matmul(x_flat, k_flat)
            y=conv_flat.reshape([mb_size, xh, xw, ychn])+pm['b']
            
        elif act == 'A':
            y= self.activate(y, hconfig)
            relu_y = y
            
        elif act =='B':
            y, aux_bn = self.forward_batch_normal_layer(y, None, pm['bn'])
            
    y, aux_stride = stride_filter(hconfig, True, y)
    
    if self.need_maps: self. maps.append(y)
        
    return y, [x_flat, k_flat, x, relu_y, aux_bn, aux_stride]


CnnExtModel.forward_conv_layer = cnn_ext_forward_conv_layer


In [ ]:
def cnn_ext_alloc_max_layer(self, input_shape, hconfig):
    ychn, xh, xw = input_shape 
    output_shape = eval_stride_shape(hconfig, False, ychn, xh, xw )
    kh, kw, sh, sw, padding = get_shape_params(hconfig, False)
    
    if padding =='VALID':padding='valid'
    if padding =='SAME':padding='same'
        
    pm=nn.MaxPool2d((kh,kw),stride=(sh,sw),padding=padding)
    
    return [pm], output_shape


CnnExtModel.alloc_max_layer = cnn_ext_alloc_max_layer

In [ ]:
def cnn_ext_forward_max_layer(self, x, hconfig, pm):
    
    mb_size, xh, xw, chn = x.shape
    
    sh, sw = get_conf_param_2d(hconfig, 'strid', [1,1])
    kh, kw = get_conf_param_2d(hconfig, 'ksize', [sh, sw])
    padding = get_conf_param(hconfig, 'padding', 'SAME')
    
    if [sh, sw] == [kh, kw] and xh % sh == 0 and xw % sw == 0 and \
    padding == 'SAME':
        return super(CnnExtModel, self).forward_max_layer(x, hconfig, pm)
    
    x_flat = get_ext_regions(x, kh, kw, -np.inf)
    x_flat = x_flat.transpose([2, 5, 0, 1, 3, 4])
    x_flat = x_flat.reshape(mb_size * chn * xh * xw, kh*kw)
    
    max_idx = np.argmax(x_flat, axis = 1)
    y = x_flat[np.arange(x_flat.shape[0]), max_idx]
    y = y.reshape([mb_size, chn, xh, xw])
    y = y.transpose([0,2,3,1])
    
    
    y, aux_stride = stride_filter(hconfig, False, y)
    
    if self.need_maps : self.maps.append(y)
    
    return y, [x.shape, kh, kw, sh, sw, padding, max_idx, aux_stride]



CnnExtModel.forward_max_layer = cnn_ext_forward_max_layer

In [ ]:
def eval_stride_shape(hconfig, conv_type, ychn, xh, xw):
    kh, kw, sh, sw, padding = get_shape_params(hconfig, conv_type)
    
    if padding == 'VALID':
        xh = xh - kh + 1
        xw = xw - kw + 1
        
    yh = xh // sh
    yw = xw // sw
    
    return [ychn, yh, yw]

In [ ]:
def stride_filter(hconfig, conv_type, y):
    
    _, xh, xw, _ = x_shape=y.shape
    nh, nw = xh, xw
    kh, kw, sh, sw, padding = get_shape_params(hconfig, conv_type)
    
    
    if padding == 'VALID':
        bh, bw = (kh -1)//2, (kw-1)//2
        nh, nw = xh - kh +1, xw - kw +1
        y=y[:,bh:bh+nh, bw:bw+nw:, :]
        
        
    if sh != 1 or sw != 1:
        bh, bw = (sh-1)//2, (sw -1)//2
        mh,mw = nh // sh, nw//sw
        y = y[:,bh:bh+mh*sh:sh, bw:bw+mw*sw:sw, :]
        
        
    return y, [x_shape, nh, nw]

In [ ]:
def get_shape_params(hconfig, conv_type):
    if conv_type:
        kh, kw = get_conf_param_2d(hconfig, 'ksize')
        sh, sw = get_conf_param_2d(hconfig, 'stride',[1,1])
        
    else:
        sh,sw = get_conf_param_2d(hconfig, 'stride', [1,1])
        kh,kw = get_conf_param_2d(hconfig, 'ksize',[sh, sw])
        
    padding=get_conf_param(hconfig, 'padding', 'SAME')
#     print(padding)
    return kh, kw, sh, sw, padding


In [ ]:
# def add_module(self,pm,name):
    
#     layer_type = get_layer_type(hconfig)
    
#     if name